In [1]:
import math
import networkx.drawing.layout
import IPython.display

In [2]:
from pikov import pikov
gamekitty = pikov.Pikov.open('gamekitty.pikov')

In [ ]:
gamekitty

In [3]:
stand_waggle = gamekitty.get_frame('stand_waggle_0')
stand_waggle

In [4]:
stand_waggle.transitions[0]

In [5]:
stand_waggle.transitions[0].target

In [6]:
gamekitty.get_image('md5-9b084a2f4cc7a7f940fac47ebe710c45')

In [7]:
#sit = gamekitty.get_clip('sit')
#sit_to_stand = gamekitty.get_clip('sit_to_stand')
#sit_paw = gamekitty.get_clip('sit_paw')
#stand_to_sit = gamekitty.get_clip('stand_to_sit')
#sit + sit_to_stand + clip + clip + stand_to_sit + sit_paw

In [3]:
frames = list(gamekitty.list_frames())
transitions = list(gamekitty.list_transitions())
graph = gamekitty.to_networkx()

In [22]:
# LINE_WIDTH = 0.006
# IMAGE_SIZE = 0.04
LINE_WIDTH = 0.03
IMAGE_SIZE = 0.3
LINE_COLOR = '#83769c'  # 'lightblue'
ARROW_WIDTH = LINE_WIDTH * 4
ARROW_POINT = ARROW_WIDTH / 3.0

# Radius to the corner of the image.
IMAGE_RADIUS = (IMAGE_SIZE / 2.0) * math.sqrt(2.0)
IMAGE_RADIUS += 0.01  # A little slush.
VIEWBOX_COORDS = -1.0 - IMAGE_RADIUS
VIEWBOX_SIZE = 2.0 * abs(VIEWBOX_COORDS)


def arrow_head(x, y, angle):
    x_start = x + (
        -1.0 * math.cos(angle) * ARROW_POINT +
        -1.0 * math.sin(angle) * ARROW_WIDTH / 2.0
    )
    x_end = x + (
        -1.0 * math.cos(angle) * ARROW_POINT +
        math.sin(angle) * ARROW_WIDTH / 2.0
    )
    y_start = y + (
        math.cos(angle) * ARROW_WIDTH / 2.0 +
        -1.0 * math.sin(angle) * ARROW_POINT
    )
    y_end = y + (
        -1.0 * math.cos(angle) * ARROW_WIDTH / 2.0 +
        -1.0 * math.sin(angle) * ARROW_POINT
    )
    return (
        f'<path d="M {x_start} {y_start} L {x} {y} {x_end} {y_end}" '
        f'stroke="{LINE_COLOR}" stroke-width="{LINE_WIDTH}" fill="none" />'
    )


def to_svg(layout, frames, transitions):
    lines = []
    for transition in transitions:
        if transition.source != transition.target:
            x1, y1 = layout[transition.source]
            x2, y2 = layout[transition.target]
            angle = math.atan2(y2 - y1, x2 - x1)
            #xo = (IMAGE_SIZE / 2.0 + 0.01) * math.cos(angle)
            #yo = (IMAGE_SIZE / 2.0 + 0.01) * math.sin(angle)
            xo = math.cos(angle) * IMAGE_RADIUS
            yo = math.sin(angle) * IMAGE_RADIUS
            line = (
                f'<line x1="{x1 + xo}" y1="{y1 + yo}" x2="{x2 - xo}" y2="{y2 - yo}" '
                f'stroke="{LINE_COLOR}" stroke-width="{LINE_WIDTH}" />'
            )
            line += arrow_head(x2 - xo, y2 - yo, angle)
        else:
            x, y = layout[transition.source]
            line = (
                f'<path d="M {x} {y - IMAGE_RADIUS} '
                f'A {IMAGE_RADIUS} {IMAGE_RADIUS} 0 0 0 {x - 2 * IMAGE_RADIUS} {y - IMAGE_RADIUS} '
                f'{IMAGE_RADIUS} {IMAGE_RADIUS} 0 0 0 {x - IMAGE_RADIUS} {y}" '
                f'stroke="{LINE_COLOR}" stroke-width="{LINE_WIDTH}" fill="none" />'
            )
            line += arrow_head(x - IMAGE_RADIUS, y, 0)
        lines.append(line)
    images = []
    for frame in frames:
        x, y = layout[frame]
        imgurl = frame.image._to_data_url()
        image = (
            f'<image href="{imgurl}" image-rendering="optimizeSpeed" '
            f'x="{x - IMAGE_SIZE / 2}" y="{y - IMAGE_SIZE / 2}" '
            f'height="{IMAGE_SIZE}" width="{IMAGE_SIZE}" />'
        )
        images.append(image)
    linesstr = '\n'.join(lines)
    imagesstr = '\n'.join(images)
    return f"""
    <svg viewBox="{VIEWBOX_COORDS} {VIEWBOX_COORDS} {VIEWBOX_SIZE} {VIEWBOX_SIZE}" xmlns="http://www.w3.org/2000/svg">
    {linesstr}
    {imagesstr}
    </svg>
    """

In [23]:
sit_nodes = [gamekitty.get_frame('sit_0')] + [
    gamekitty.get_frame(f'sit_to_stand_{index}') for index in range(4)
] + [
    gamekitty.get_frame(f'stand_waggle_{index}') for index in range(2)
] + [
    gamekitty.get_frame(f'stand_to_sit_{index}') for index in range(5)
]
paw_nodes = [gamekitty.get_frame(f'sit_paw_{index}') for index in range(3, -1, -1)]
shell_layout = networkx.drawing.layout.shell_layout(graph, [paw_nodes, sit_nodes])
IPython.display.display(IPython.display.SVG(data=to_svg(shell_layout, frames, transitions)))